#**PREPROCESAMIENTO Y NORMALIZACIÓN EN VAL BAJO TRAIN**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
## Cargar datos fd_val
fd_val = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val.csv", sep=';',)

print(f"Registros en fd_val: {len(fd_val)}")

Registros en fd_val: 376


In [ ]:
fd_val.shape

(376, 72)

##Normalización y redimensionado de datos numéricos y categóricos DATOS TABULARES fd_val basado en TRAIN

In [ ]:
#

# Definir las columnas que no aportan información relevante
not_useful_columns = [
    'first_review', 'last_review', 'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms',
    'calculated_host_listings_count', 'license', 'calendar_last_scraped',
    'has_availability', 'availability_30', 'availability_60', 'availability_90',
    'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'host_listings_count',
    'host_neighbourhood', 'host_picture_url', 'host_about', 'host_location',
    'host_thumbnail_url', 'host_name', 'host_url', 'host_id', 'source',
    'last_scraped', 'scrape_id', 'neighbourhood_group_cleansed', 'calendar_updated'
]

# Hacemos un drop de todas las columnas que no aportan información relevante a precio o inmueble en fd_val
fd_val.drop(not_useful_columns, axis=1, inplace=True)

fd_val.shape

(376, 41)

##nulls en review_scores_* y reviews_per_month

In [ ]:
import pandas as pd
import joblib

# Definir las columnas a imputar
review_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                  'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                  'review_scores_value', 'reviews_per_month']

# Cargar las medianas previamente guardadas del archivo
medianas_reviews = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/medianas_reviews.pkl')

# Imputar valores nulos en el conjunto de validación con las medianas
for column in review_columns:
    fd_val[column] = fd_val[column].fillna(medianas_reviews[column])

# Imprimir el número de nulos restantes después de la imputación
print("Número de nulos restantes después de la imputación en val:")
print(fd_val[review_columns].isnull().sum())


Número de nulos restantes después de la imputación en val:
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
reviews_per_month              0
dtype: int64


##Imputacion 'bathrooms' y 'bathrooms_text'

In [ ]:
import re
import pandas as pd
import joblib

# Cargar la información de imputación previamente guardada
bathrooms_info = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/bathrooms_info.pkl')
bathrooms_median = bathrooms_info['bathrooms_median']
porcentaje_bathrooms_shared = bathrooms_info['porcentaje_bathrooms_shared']

# Extraemos el número de baños de bathrooms_text
def extract_bathrooms(text):
    if pd.isnull(text):
        return None
    match = re.search(r'(\d+)', text)
    return float(match.group(1)) if match else None

# Crear una nueva columna 'bathrooms_shared' para indicar si el baño es compartido o no
fd_val['bathrooms_shared'] = fd_val['bathrooms_text'].apply(lambda x: 1 if 'shared' in str(x).lower() else 0)

# Imputar bathrooms usando bathrooms_text cuando bathrooms está en nulo
fd_val['bathrooms'] = fd_val['bathrooms'].combine_first(fd_val['bathrooms_text'].apply(extract_bathrooms))

# Imputar valores faltantes restantes con la mediana de 'bathrooms' calculada del train
fd_val['bathrooms'] = fd_val['bathrooms'].fillna(bathrooms_median)

# Imputar valores faltantes restantes en 'bathrooms_shared' usando el porcentaje calculado
fd_val['bathrooms_shared'] = fd_val['bathrooms_shared'].fillna(round(porcentaje_bathrooms_shared))

# Eliminar bathrooms_text si ya no tiene nulos en bathrooms y bathrooms_shared
if fd_val['bathrooms'].isnull().sum() == 0 and fd_val['bathrooms_shared'].isnull().sum() == 0:
    fd_val.drop(columns=['bathrooms_text'], inplace=True)
    print("'bathrooms_text' ha sido eliminada porque ya no hay nulos en 'bathrooms' y 'bathrooms_shared'.")
else:
    print("'bathrooms_text' no ha sido eliminada porque aún hay nulos en 'bathrooms' o 'bathrooms_shared'.")

# Imprimir el número de nulos restantes después de la imputación
print("Nulos en 'bathrooms' después de la imputación en val:", fd_val['bathrooms'].isnull().sum())
print("Nulos en 'bathrooms_shared' en val:", fd_val['bathrooms_shared'].isnull().sum())

'bathrooms_text' ha sido eliminada porque ya no hay nulos en 'bathrooms' y 'bathrooms_shared'.
Nulos en 'bathrooms' después de la imputación en val: 0
Nulos en 'bathrooms_shared' en val: 0


# Imputar 'beds'

In [ ]:
import pandas as pd
import joblib

# Definir las columnas a imputar
group_columns = ['property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_shared']

# Cargar las medianas previamente guardadas del archivo
median_beds_grouped = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/median_beds_grouped.pkl')
mediana_global_beds = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/mediana_global_beds.pkl')

# Imputar valores nulos en el conjunto de validación con las medianas calculadas por grupo
fd_val = fd_val.merge(median_beds_grouped, on=group_columns, how='left')
fd_val['beds'] = fd_val['beds'].fillna(fd_val['beds_median_grouped'])

# Imputar cualquier valor restante con la mediana global de 'beds'
fd_val['beds'] = fd_val['beds'].fillna(mediana_global_beds)

# Eliminar la columna temporal 'beds_median_grouped' después de la imputación
fd_val.drop(columns=['beds_median_grouped'], inplace=True)

# Imprimir el número de nulos restantes después de la imputación
print("Nulos restantes en 'beds' después de la imputación en val:", fd_val['beds'].isnull().sum())

Nulos restantes en 'beds' después de la imputación en val: 0


# Imputar 'bedrooms'


In [ ]:
import numpy as np
import pandas as pd
from difflib import get_close_matches

# Definir los valores conocidos de 'room_type'
known_room_types = ["Entire home/apt", "Private room", "Hotel room", "Shared room"]

# Función de imputación de 'bedrooms' basada en reglas mejoradas (misma que en train)
def imputar_bedrooms(df):
    for index, row in df.iterrows():
        room_type = row['room_type']

        # Normalizar valor: Convertir a minúsculas para evitar problemas de mayúsculas/minúsculas
        room_type_normalized = room_type.lower() if pd.notnull(room_type) else None

        # Intentar encontrar coincidencias aproximadas con los valores conocidos
        match = get_close_matches(room_type_normalized, [rt.lower() for rt in known_room_types], n=1, cutoff=0.8)

        # Si encontramos una coincidencia, utilizamos el valor conocido
        if match:
            room_type_final = match[0].capitalize()
        else:
            # Si no hay coincidencias, intentamos hacer una clasificación basada en accommodates
            accommodates = row['accommodates']
            if pd.notnull(accommodates):
                if accommodates <= 2:
                    room_type_final = "Private room"  # Suposición razonable si solo tiene capacidad para 2
                elif accommodates >= 3 and accommodates <= 6:
                    room_type_final = "Entire home/apt"  # Suposición razonable para acomodar 3 a 6 personas
                else:
                    room_type_final = "Entire home/apt"  # Suposición razonable para más de 6 personas
            else:
                # Si no podemos hacer una clasificación basada en accommodates, asignamos un valor por defecto
                room_type_final = "Unknown"

        # Aplicar reglas para imputar 'bedrooms'
        if room_type_final == "Entire home/apt":
            # Reglas para 'Entire home/apt'
            if row['accommodates'] <= 2:
                df.at[index, 'bedrooms'] = 1
            elif 3 <= row['accommodates'] <= 4:
                df.at[index, 'bedrooms'] = 2
            elif 5 <= row['accommodates'] <= 6:
                df.at[index, 'bedrooms'] = 3
            else:
                df.at[index, 'bedrooms'] = np.ceil(row['accommodates'] / 2)

        elif room_type_final == "Private room":
            # Regla para 'Private room'
            df.at[index, 'bedrooms'] = 1

        elif room_type_final in ["Hotel room", "Shared room"]:
            # Regla para 'Hotel room' y 'Shared room'
            df.at[index, 'bedrooms'] = 1

        elif room_type_final == "Unknown":
            # Valor predeterminado para un 'room_type' desconocido, basándose en el número de accommodates
            df.at[index, 'bedrooms'] = np.ceil(row['accommodates'] / 2) if pd.notnull(row['accommodates']) else 1

    return df

# Aplicar la función a fd_val
fd_val = imputar_bedrooms(fd_val)

# Verificar nulos restantes en 'bedrooms' después de la imputación en val
print("Nulos restantes en 'bedrooms' después de la imputación en fd_val:", fd_val['bedrooms'].isnull().sum())

Nulos restantes en 'bedrooms' después de la imputación en fd_val: 0


In [ ]:
# Identificar columnas que contienen la palabra 'host' en su nombre
host_columns = [col for col in fd_val.columns if 'host' in col]

# Mostrar valores nulos en estas columnas
null_counts = fd_val[host_columns].isnull().sum()

print("Valores nulos en las columnas que contienen 'host':")
print(null_counts)


Valores nulos en las columnas que contienen 'host':
host_since                    0
host_response_time           31
host_response_rate           31
host_acceptance_rate         26
host_is_superhost             8
host_total_listings_count     0
host_verifications            0
host_has_profile_pic          0
host_identity_verified        0
dtype: int64


###Imputacion de NULLS columnas catégoricas:
Imputar columnas host_

In [ ]:
import joblib
import numpy as np
import pandas as pd

# Cargar la información guardada de train
host_info = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/host_info.pkl')
mean_host_response_rate = host_info['mean_host_response_rate']
mean_host_acceptance_rate = host_info['mean_host_acceptance_rate']
mode_host_is_superhost = host_info['mode_host_is_superhost']

# Imputar 'host_is_superhost' con el valor más frecuente (moda) en el dataset
fd_val['host_is_superhost'] = fd_val['host_is_superhost'].fillna(mode_host_is_superhost)

# Convertir 'host_response_rate' y 'host_acceptance_rate' de texto a valores numéricos quitando el % y dividiendo el número por 100
if fd_val['host_response_rate'].dtype == 'O':
    fd_val['host_response_rate'] = fd_val['host_response_rate'].str.rstrip('%').astype(float) / 100
if fd_val['host_acceptance_rate'].dtype == 'O':
    fd_val['host_acceptance_rate'] = fd_val['host_acceptance_rate'].str.rstrip('%').astype(float) / 100

# Imputar 'host_response_rate' y 'host_acceptance_rate' basado en la media dentro de los grupos de 'host_is_superhost'
fd_val['host_response_rate'] = fd_val.apply(
    lambda row: mean_host_response_rate.get(row['host_is_superhost'], np.nan) if pd.isnull(row['host_response_rate']) else row['host_response_rate'], axis=1)
fd_val['host_acceptance_rate'] = fd_val.apply(
    lambda row: mean_host_acceptance_rate.get(row['host_is_superhost'], np.nan) if pd.isnull(row['host_acceptance_rate']) else row['host_acceptance_rate'], axis=1)

# Imputar 'host_response_time' basado en reglas condicionales en función de 'host_is_superhost' y 'host_response_rate'
def imputar_host_response_time(row):
    if pd.isnull(row['host_response_time']):
        if row['host_is_superhost'] == 't' or (row['host_response_rate'] is not None and row['host_response_rate'] >= 0.9):
            return 'within an hour'
        elif row['host_response_rate'] is not None and 0.5 <= row['host_response_rate'] < 0.9:
            return 'within a day'
        else:
            return 'a few days or more'
    return row['host_response_time']

# Aplicar la imputación condicional a 'host_response_time'
fd_val['host_response_time'] = fd_val.apply(imputar_host_response_time, axis=1)

# Verificar nulos restantes en las columnas host_*
print("Nulos restantes en host_response_time (val):", fd_val['host_response_time'].isnull().sum())
print("Nulos restantes en host_response_rate (val):", fd_val['host_response_rate'].isnull().sum())
print("Nulos restantes en host_acceptance_rate (val):", fd_val['host_acceptance_rate'].isnull().sum())
print("Nulos restantes en host_is_superhost (val):", fd_val['host_is_superhost'].isnull().sum())


Nulos restantes en host_response_time (val): 0
Nulos restantes en host_response_rate (val): 0
Nulos restantes en host_acceptance_rate (val): 0
Nulos restantes en host_is_superhost (val): 0


##NULLS 'neighbourhood', primero CODIFICACIÓN

In [ ]:
# Verificar las categorías únicas en 'neighbourhood_cleansed'
unique_neighbourhood_cleansed = fd_val['neighbourhood_cleansed'].unique()
print(unique_neighbourhood_cleansed)
print(f'Número total de categorías únicas en neighbourhood_cleansed: {len(unique_neighbourhood_cleansed)}')

['Downtown' 'Charlestown' 'West Roxbury' 'South End' 'Fenway' 'Roslindale'
 'Roxbury' 'Beacon Hill' 'Jamaica Plain' 'Dorchester' 'Brighton'
 'Back Bay' 'Mattapan' 'South Boston' 'North End' 'Allston' 'Mission Hill'
 'East Boston' 'Hyde Park' 'Bay Village' 'West End'
 'South Boston Waterfront' 'Chinatown' 'Leather District']
Número total de categorías únicas en neighbourhood_cleansed: 24


In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder

# Cargar el LabelEncoder y los centroides previamente guardados
le_combined = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/label_encoder_combined.pkl")
centroides = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/centroides_neighbourhood.pkl")

# Definición de la función haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

# Limpieza y estandarización de las columnas 'neighbourhood' y 'neighbourhood_cleansed' en validación
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.strip()
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.replace(r"\s+", " ", regex=True)
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.replace(r", ,", ",", regex=True)
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.replace(r"Dorchster", "Dorchester", regex=False)
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.replace(r"\s+,", ",", regex=True)
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.replace(r"\(.*\)", "", regex=True).str.strip()
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.replace(r", Massachusetts, United States", "", regex=False)
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.strip()
fd_val['neighbourhood'] = fd_val['neighbourhood'].str.replace(r"Jamaica Plain, Boston", "Jamaica Plain", regex=False)

fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.strip()
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.replace(r"\s+", " ", regex=True)
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.replace(r", ,", ",", regex=True)
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.replace(r"Dorchster", "Dorchester", regex=False)
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.replace(r"\s+,", ",", regex=True)
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.replace(r"\(.*\)", "", regex=True).str.strip()
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.replace(r", Massachusetts, United States", "", regex=False)
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].str.strip()

# Manejar valores desconocidos para 'neighbourhood' usando los centroides
def imputar_barrio_mas_cercano(row, centroides):
    if row['neighbourhood'] not in le_combined.classes_:
        # Calcular distancias geográficas a los centroides y asignar el barrio más cercano
        distancias = centroides.apply(lambda x: haversine(row['latitude'], row['longitude'], x['latitude'], x['longitude']), axis=1)
        barrio_mas_cercano = distancias.idxmin()
        return barrio_mas_cercano
    return row['neighbourhood']

# Aplicar la imputación para barrios desconocidos
fd_val['neighbourhood'] = fd_val.apply(imputar_barrio_mas_cercano, axis=1, args=(centroides,))

# Codificar las columnas utilizando el LabelEncoder
fd_val['neighbourhood'] = fd_val['neighbourhood'].apply(
    lambda x: le_combined.transform([x])[0] if x in le_combined.classes_ else le_combined.transform(["<UNK>"])[0]
)
fd_val['neighbourhood_cleansed'] = fd_val['neighbourhood_cleansed'].apply(
    lambda x: le_combined.transform([x])[0] if x in le_combined.classes_ else le_combined.transform(["<UNK>"])[0]
)

print("Valores únicos resultantes de 'neighbourhood' y 'neighbourhood_cleansed' en validación:",
      pd.concat([fd_val['neighbourhood'], fd_val['neighbourhood_cleansed']]).unique())


Valores únicos resultantes de 'neighbourhood' y 'neighbourhood_cleansed' en validación: [12  5 21 14 10  6  7 15  9 11 27 25 13 22  4  2 18 23 20  1 19  3 26 24
  8 16]


##Imputación 'neighbourhood'

In [ ]:
from sklearn.impute import KNNImputer

import joblib

# Cargar el imputador KNN previamente ajustado en el conjunto de entrenamiento
imputer = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/knn_imputer_trained.pkl")

# Definir las columnas que se utilizarán para la imputación
columns_for_imputation = ['latitude', 'longitude', 'neighbourhood', 'neighbourhood_cleansed']

# Aplicar el imputador al conjunto de validación
fd_val_imputed = imputer.transform(fd_val[columns_for_imputation])

# Actualizar el DataFrame con los valores imputados en 'neighbourhood'
fd_val['neighbourhood'] = fd_val_imputed[:, 2]

# Guardar el dataset de validación imputado si es necesario
fd_val.to_csv('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_imputed.csv', sep=';', index=False)

In [ ]:
# Identificar columnas categóricas con valores nulos
categorical_columns = fd_val.select_dtypes(include=['object']).columns
categorical_null_columns = categorical_columns[fd_val[categorical_columns].isnull().any()]

# Contar los valores nulos en cada una de las columnas categóricas con nulos
categorical_null_counts = fd_val[categorical_null_columns].isnull().sum()
print(categorical_null_counts)


Series([], dtype: float64)


##Ya no tenemos NULLS ahora convertiremos las categoricas a numericas restantes:

In [ ]:
#cargar dataset fd_val_imputed
fd_val_imputed = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_imputed.csv", sep=';')

##Comenzaré con las categorias host_

In [ ]:
# Convertir la columna 'host_since' a datetime para crear la columna 'years_being_host' que indicaria la antiguedad del host
from datetime import datetime

fd_val_imputed['host_since'] = fd_val_imputed['host_since'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
fd_val_imputed['years_being_host'] = fd_val_imputed['host_since'].apply(lambda x: datetime.now().year - x.year)

fd_val_imputed.drop(['host_since'], axis=1, inplace=True)

print(fd_val_imputed['years_being_host'].dtype)

int64


##'host_response_time' valores unicos

In [ ]:
# Mapping para convertir los valores de 'host_response_time' en valores numéricos relevancia
response_time_mapping = {
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1
}

# Aplicar la transformación a la columna 'host_response_time'
fd_val_imputed['host_response_time'] = fd_val_imputed['host_response_time'].map(response_time_mapping)

# Verificar los valores únicos después de la transformación
print(fd_val_imputed['host_response_time'].unique())

[4 3 2 1]


##'host_is_superhost' ,'host_has_profile_pic', 'host_identity_verified' e 'instant_bookable'

In [ ]:
# Convertir las cadenas 't' y 'f' a valores numéricos binarios (0 y 1) usando .apply()

boolean_columns = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
for col in boolean_columns:
    fd_val_imputed[col] = fd_val_imputed[col].apply(lambda x: 1 if x == 't' else 0)

# Verificar los valores únicos para confirmar la transformación
for col in boolean_columns:
    print(f"Valores únicos en {col} (val):", fd_val_imputed[col].unique())

Valores únicos en host_is_superhost (val): [0 1]
Valores únicos en host_has_profile_pic (val): [1 0]
Valores únicos en host_identity_verified (val): [1 0]
Valores únicos en instant_bookable (val): [0 1]


##'host_verifications'

In [ ]:
print("Valores únicos en 'host_verifications':")
print(fd_val_imputed['host_verifications'].unique())


Valores únicos en 'host_verifications':
["['email', 'phone']" "['phone']" "['email', 'phone', 'work_email']"
 "['phone', 'work_email']"]


In [ ]:
from ast import literal_eval
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargar el escalador previamente entrenado en train
scaler = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_verifications.pkl')

# Convertir las cadenas de texto en listas reales usando literal_eval
fd_val_imputed['host_verifications'] = fd_val_imputed['host_verifications'].apply(literal_eval)

# Normalizar las listas eliminando espacios innecesarios y asegurando un formato consistente
fd_val_imputed['host_verifications'] = fd_val_imputed['host_verifications'].apply(
    lambda x: [item.strip().lower() for item in x]
)

# Contar el número de verificaciones para cada anfitrión, asegurando que las listas vacías sean tratadas como 0
fd_val_imputed['num_host_verifications'] = fd_val_imputed['host_verifications'].apply(
    lambda x: len(x) if len(x) > 0 else 0  # Listas vacías -> 0
)

# Clipping en el rango esperado (0 a 3) definido en train
fd_val_imputed['num_host_verifications'] = np.clip(
    fd_val_imputed['num_host_verifications'], scaler.data_min_[0], scaler.data_max_[0]
)

# Transformar los valores usando el escalador de train
fd_val_imputed['num_host_verifications'] = scaler.transform(
    fd_val_imputed[['num_host_verifications']]
)

# Eliminar la columna original 'host_verifications' para evitar redundancia
fd_val_imputed.drop('host_verifications', axis=1, inplace=True)

# Verificar las primeras filas para asegurarnos de que la transformación se realizó correctamente
print(fd_val_imputed[['num_host_verifications']].head())


   num_host_verifications
0                    0.50
1                    0.50
2                    0.50
3                    0.01
4                    0.50


##'property_type'

In [ ]:
import joblib

# Cargar las medias calculadas para 'property_type' desde el conjunto de entrenamiento
property_price_means = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/property_price_means.pkl")

# Crear una nueva columna 'property_type_encoded' en el conjunto de validación
fd_val_imputed['property_type_encoded'] = fd_val_imputed['property_type'].map(property_price_means)

# Manejar valores no vistos en el diccionario, asignándoles la media global calculada en el conjunto de entrenamiento
global_mean_price_train = sum(property_price_means.values()) / len(property_price_means)
fd_val_imputed['property_type_encoded'] = fd_val_imputed['property_type_encoded'].fillna(global_mean_price_train)

# Eliminar la columna original 'property_type'
fd_val_imputed.drop('property_type', axis=1, inplace=True)

# Verificar las primeras filas para asegurarnos de que la transformación se realizó correctamente
print(fd_val_imputed[['property_type_encoded']].head())


   property_type_encoded
0             190.545852
1             242.500000
2              81.504950
3             190.545852
4             190.545852


##'room_type'

In [ ]:
import joblib

# Cargar las medias calculadas previamente y la media global desde el entrenamiento
room_data = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/room_price_means.pkl")
room_price_means = room_data['room_price_means']
global_mean_price_train = room_data['global_mean_price_train']

# Aplicar el promedio calculado en train para codificar la columna en val
fd_val_imputed['room_type_encoded'] = fd_val_imputed['room_type'].map(room_price_means)

# Manejar valores no vistos en train asignándoles la media global
fd_val_imputed['room_type_encoded'] = fd_val_imputed['room_type_encoded'].fillna(global_mean_price_train)

# Eliminar la columna original 'room_type' para evitar redundancia
fd_val_imputed.drop('room_type', axis=1, inplace=True)

# Verificar las primeras filas para asegurarnos de que la transformación se realizó correctamente
print(fd_val_imputed[['room_type_encoded']].head())

   room_type_encoded
0         217.807606
1         217.807606
2         129.135593
3         217.807606
4         217.807606


In [ ]:
# Eliminar columnas innecesarias del conjunto val
fd_val_imputed.drop(columns=['listing_url', 'picture_url', 'amenities', 'id'], inplace=True)

# Verificar que las columnas han sido eliminadas
print("Columnas restantes en fd_val:", fd_val_imputed.columns)

Columnas restantes en fd_val: Index(['host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month',
       'bathrooms_shared', 'years_being_host', 'num_host_verifications',
       'property_type_encoded', 'room_type_encoded'],
      dtype='object')


In [ ]:
print(len(fd_val_imputed))

376


In [ ]:
# Verificar si existen columnas no numéricas en fd_train_imputed
non_numeric_columns = fd_val_imputed.select_dtypes(exclude=['number']).columns

# Imprimir las columnas no numéricas
print("Columnas no numéricas en fd_val_imputed:", non_numeric_columns)


Columnas no numéricas en fd_val_imputed: Index([], dtype='object')


In [ ]:
# Guardar el conjunto val imputado y con todas las columnas convertidas a numéricas
fd_val_imputed.to_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_imputed_num.csv", sep=';', index=False)

# Confirmación del guardado
print("El archivo CSV con los datos imputados y numéricos ha sido guardado como 'fd_val_imputed_num.csv'")

El archivo CSV con los datos imputados y numéricos ha sido guardado como 'fd_val_imputed_num.csv'


In [ ]:
# Cargar el conjunto de validación imputado y numérico desde el archivo CSV
fd_val_imputed_num = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_imputed_num.csv", sep=';')

# Confirmación de la carga
print("El archivo CSV 'fd_val_imputed_num.csv' ha sido cargado exitosamente.")

El archivo CSV 'fd_val_imputed_num.csv' ha sido cargado exitosamente.


##OUTLIERS
"price"

In [ ]:
# Número de registros antes de eliminar outliers inconsistentes en validación
initial_count_val = fd_val_imputed_num.shape[0]

# Definir un umbral para el precio extremo basado en entrenamiento
price_threshold = 3000

# Filtrar registros inconsistentes con precio mayor a $3,000 y características poco justificadas en validación
inconsistent_outliers_val = fd_val_imputed_num[
    (fd_val_imputed_num['price'] > price_threshold) &
    (fd_val_imputed_num['accommodates'] <= 2) &
    (fd_val_imputed_num['beds'] <= 2)
]

# Número de registros inconsistentes que se eliminarán en validación
outliers_count_val = inconsistent_outliers_val.shape[0]

# Eliminar esos registros inconsistentes del dataset de validación
fd_val_imputed_num = fd_val_imputed_num.drop(inconsistent_outliers_val.index)

# Número de registros restantes después de eliminar los outliers en validación
final_count_val = fd_val_imputed_num.shape[0]

# Mostrar cuántos registros se eliminaron en validación
print(f"Registros eliminados en validación: {outliers_count_val}")
print(f"Registros restantes después de eliminar outliers inconsistentes en validación: {final_count_val}")

Registros eliminados en validación: 0
Registros restantes después de eliminar outliers inconsistentes en validación: 376


##Transformación logarítmica en 'price'

In [ ]:
import numpy as np

# Aplicar la transformación logarítmica con `np.log1p` a la columna 'price'
fd_val_imputed_num['price_log'] = np.log1p(fd_val_imputed_num['price'])

# Verificar la transformación
print(fd_val_imputed_num[['price', 'price_log']].head())

# Eliminar la columna original 'price' para evitar problemas de colinealidad
fd_val_imputed_num.drop(columns=['price'], inplace=True)


   price  price_log
0  241.0   5.488938
1  399.0   5.991465
2   85.0   4.454347
3   94.0   4.553877
4   91.0   4.521789


Capping a 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_reviews', 'reviews_per_month', 'host_total_listings_count', 'minimum_nights', 'maximum_nights'

In [ ]:
import numpy as np
import pickle

# Cargar los thresholds calculados previamente en train
with open('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/capping_thresholds.pkl', 'rb') as f:
    capping_thresholds = pickle.load(f)

# Aplicar capping a las columnas en el conjunto de validación
for column in ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_reviews', 'reviews_per_month', 'host_total_listings_count', 'minimum_nights', 'maximum_nights']:
    # Obtener el threshold para la columna
    threshold = capping_thresholds[column]

    # Aplicar capping en los valores que superan el threshold
    fd_val_imputed_num[column] = np.where(fd_val_imputed_num[column] > threshold, threshold, fd_val_imputed_num[column])

# Verificar algunos valores después del capping en fd_val
print("Valores después del capping en 'maximum_nights' (val):")
print(fd_val_imputed_num['maximum_nights'].sort_values(ascending=False).head())


Valores después del capping en 'maximum_nights' (val):
255    1125.0
224    1125.0
77     1125.0
284    1125.0
164    1125.0
Name: maximum_nights, dtype: float64


In [ ]:
# Guardar el fd_val_imputed_num_out preprocesado CON TRABAJO DE OUTLIERS en un archivo CSV
fd_val_imputed_num.to_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_imputed_num_out.csv", sep=';', index=False)

In [ ]:
# Cargar el DataFrame preprocesado preprocesado CON TRABAJO DE OUTLIERS desde un archivo CSV
import pandas as pd
fd_val_imputed_num_out = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_imputed_num_out.csv", sep=';')

In [ ]:
fd_val_imputed_num_out.shape

(376, 37)

Eliminar 'neighbourhood por que tiene mas correlacion con 'price_log 'neighbourhood_cleansed'

In [ ]:
#drop de 'neighbourhood'
fd_val_imputed_num_out.drop(columns=['neighbourhood'], inplace=True)

In [ ]:
print(fd_val_imputed_num_out.head(6).to_string())

   host_response_time  host_response_rate  host_acceptance_rate  host_is_superhost  host_total_listings_count  host_has_profile_pic  host_identity_verified  neighbourhood_cleansed   latitude  longitude  accommodates  bathrooms  bedrooms  beds  minimum_nights  maximum_nights  minimum_minimum_nights  availability_365  number_of_reviews  number_of_reviews_ltm  number_of_reviews_l30d  review_scores_rating  review_scores_accuracy  review_scores_cleanliness  review_scores_checkin  review_scores_communication  review_scores_location  review_scores_value  instant_bookable  reviews_per_month  bathrooms_shared  years_being_host  num_host_verifications  property_type_encoded  room_type_encoded  price_log
0                   4                1.00                  0.98                  0                      171.0                     1                       1                      11  42.353230 -71.063140           6.0        1.0       3.0   2.0            29.0           365.0                      2

# **NORMALIZACIÓN**

####Normalizare 'bedrooms', 'beds', 'accommodates', 'bathrooms', que tienen valores como 1, 0 [0.01, 0.99]. Para que mas adelante no presenten problemas con funciones de activacion como sigmoid o tanh.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargar el escalador entrenado en train
scaler_bedrooms_beds_adjusted = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_bedrooms_beds_adjusted.pkl')

# Aplicar el escalador a 'bedrooms', 'beds', 'accommodates', y 'bathrooms' en validación
fd_val_imputed_num_out[['bedrooms', 'beds', 'accommodates', 'bathrooms']] = scaler_bedrooms_beds_adjusted.transform(
    fd_val_imputed_num_out[['bedrooms', 'beds', 'accommodates', 'bathrooms']]
)

# Verificar los valores después del ajuste
adjusted_bedrooms_beds_stats_val = fd_val_imputed_num_out[['bedrooms', 'beds', 'accommodates', 'bathrooms']].agg(['min', 'max'])

# Mostrar los resultados
print(adjusted_bedrooms_beds_stats_val)


     bedrooms  beds  accommodates  bathrooms
min      0.01  0.01          0.01       0.01
max      0.99  0.99          0.99       0.99


In [ ]:
print(fd_val_imputed_num_out.head().to_string())

   host_response_time  host_response_rate  host_acceptance_rate  host_is_superhost  host_total_listings_count  host_has_profile_pic  host_identity_verified  neighbourhood_cleansed   latitude  longitude  accommodates  bathrooms  bedrooms  beds  minimum_nights  maximum_nights  minimum_minimum_nights  availability_365  number_of_reviews  number_of_reviews_ltm  number_of_reviews_l30d  review_scores_rating  review_scores_accuracy  review_scores_cleanliness  review_scores_checkin  review_scores_communication  review_scores_location  review_scores_value  instant_bookable  reviews_per_month  bathrooms_shared  years_being_host  num_host_verifications  property_type_encoded  room_type_encoded  price_log
0                   4                1.00                  0.98                  0                      171.0                     1                       1                      11  42.353230 -71.063140      0.455455   0.336667     0.402  0.15            29.0           365.0                      2

#Columnas Numéricas para Normalización

In [ ]:
import numpy as np
import joblib

# Cargar configuraciones guardadas de train
columns_log_transform = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/log_transform_columns.pkl')
scaler = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_log_transform.pkl')

# Revisar valores antes de cualquier corrección
print("Valores originales en validación (val):")
print(fd_val_imputed_num_out[columns_log_transform].agg(['min', 'max']))

# Aplicar log1p (sin valores negativos)
fd_val_imputed_num_out[columns_log_transform] = fd_val_imputed_num_out[columns_log_transform].clip(lower=0).apply(np.log1p)

# Revisar valores después de log1p
print("Valores después de log1p en validación (val):")
print(fd_val_imputed_num_out[columns_log_transform].agg(['min', 'max']))

# Asegurar que los valores estén dentro del rango observado en train
for col in columns_log_transform:
    min_train = scaler.data_min_[columns_log_transform.index(col)]
    max_train = scaler.data_max_[columns_log_transform.index(col)]
    # Clipping a los rangos observados en train
    fd_val_imputed_num_out[col] = fd_val_imputed_num_out[col].clip(lower=min_train, upper=max_train)

# Aplicar el escalador para normalizar los valores
fd_val_imputed_num_out[columns_log_transform] = scaler.transform(fd_val_imputed_num_out[columns_log_transform])

# Verificar los valores finales después del escalado
print("Valores finales en validación (val):")
print(fd_val_imputed_num_out[columns_log_transform].agg(['min', 'max']))


Valores originales en validación (val):
     host_total_listings_count  minimum_nights  maximum_nights  \
min                        1.0             1.0             1.0   
max                     1436.0           115.2          1125.0   

     minimum_minimum_nights  availability_365  number_of_reviews  \
min                       1                 0               0.00   
max                     400               365             375.52   

     number_of_reviews_ltm  number_of_reviews_l30d  reviews_per_month  \
min                      0                       0             0.0100   
max                    152                      11             8.9256   

     property_type_encoded  room_type_encoded  
min              38.000000          51.833333  
max             677.809524         508.357143  
Valores después de log1p en validación (val):
     host_total_listings_count  minimum_nights  maximum_nights  \
min                   0.693147        0.693147        0.693147   
max           

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargar el escalador y las columnas seleccionadas desde train
scaler_min_max = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_min_max_train.pkl')
columns_min_max = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/columns_min_max.pkl')

# Aplicar el escalador a las columnas seleccionadas
fd_val_imputed_num_out[columns_min_max] = scaler_min_max.transform(fd_val_imputed_num_out[columns_min_max])

# Verificar las estadísticas después del escalado
print("Estadísticas después del escalado en validación (val):")
print(fd_val_imputed_num_out[columns_min_max].agg(['min', 'max']))



Estadísticas después del escalado en validación (val):
     host_response_rate  host_acceptance_rate  latitude  longitude  \
min              0.1766                  0.01  0.107317   0.028622   
max              0.9900                  0.99  0.960058   0.928220   

     review_scores_rating  review_scores_accuracy  review_scores_cleanliness  \
min                 0.255                   0.402                      0.206   
max                 0.990                   0.990                      0.990   

     review_scores_checkin  review_scores_communication  \
min                  0.402                         0.01   
max                  0.990                         0.99   

     review_scores_location  review_scores_value  years_being_host  
min                    0.01                 0.01              0.01  
max                    0.99                 0.99              0.99  


###*'neighbourhood_cleansed'* ya paso por labelencoder y esta codficada.

No necesita en este momento normalizacion, puede trabajarse mas segun el modelo que haga si es con redes neuronales o arboles, etc.

In [ ]:
# Guardar el fd_val_imputed_num_out con final de preprocesado y NORMALIZACION en un archivo CSV fd_val_finpreprocesado.
fd_val_imputed_num_out.to_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_finpreprocesado.csv", sep=';', index=False)


In [ ]:
# Cargar el fd_val_finpreprocesado.csv
import pandas as pd
fd_val_finpreprocesado = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_val_finpreprocesado.csv", sep=';')

In [ ]:
fd_val_finpreprocesado.shape

(376, 36)

In [ ]:
import pandas as pd

# Configuración para mostrar todas las columnas sin truncar
pd.set_option('display.max_columns', None)

# Para fd_val_finalpreprocesado
print("\nMínimo y máximo en cada columna de fd_val_finpreprocesado:")
min_max_val = fd_val_finpreprocesado.agg(['min', 'max'])
print(min_max_val)

# Opcional: Restaurar configuración para evitar demasiadas columnas en futuras salidas
pd.reset_option('display.max_columns')


Mínimo y máximo en cada columna de fd_val_finpreprocesado:
     host_response_time  host_response_rate  host_acceptance_rate  \
min                   1              0.1766                  0.01   
max                   4              0.9900                  0.99   

     host_is_superhost  host_total_listings_count  host_has_profile_pic  \
min                  0                       0.01                     0   
max                  1                       0.99                     1   

     host_identity_verified  neighbourhood_cleansed  latitude  longitude  \
min                       0                       1  0.107317   0.028622   
max                       1                      27  0.960058   0.928220   

     accommodates  bathrooms  bedrooms  beds  minimum_nights  maximum_nights  \
min          0.01       0.01      0.01  0.01            0.01            0.01   
max          0.99       0.99      0.99  0.99            0.99            0.99   

     minimum_minimum_nights  availab